In [51]:
import pandas as pd
import numpy as np
import os
import re
import warnings

import seaborn as sns
import matplotlib.pyplot as plt

try:
    from pypdf import PdfReader
except:
    !pip install pypdf
    from pypdf import PdfReader

try:
    import spacy
except:
    !pip install spacy
    !python -m spacy download en
    import spacy

try:
    import chardet
except:
    !pip install chardet
    import chardet
    
    
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report, mean_squared_error, f1_score

import xgboost
import catboost

from joblib import dump, load


In [52]:
warnings.filterwarnings("ignore")
plt.style.use('ggplot')
nlp = spacy.load("en_core_web_sm")

SCORES_PATH    = 'English_level/English_scores'
SUBTITLES_PATH = 'English_level/English_scores/Subtitles_all'
OXFORD_PATH    = 'English_level/Oxford_CEFR_level'

ENGLISH_LEVELS = ['A1', 'A2', 'B1', 'B2', 'C1']

RANDOM_STATE = 7

# LOAD DATA

## The Oxford core by CEFR level

In [53]:
def process_pdf(content):
    
    current_level = ''
    words_by_level = {key:[] for key in ENGLISH_LEVELS}
    
    for line in content:
        line = line.strip()

        if 'Oxford' in line or 'English' in line:
            pass
        
        elif line in ENGLISH_LEVELS:
            level = line
            
        elif ' ' in line:
            line = re.sub(r'\d|,', '', line.lower())   # remove digits and commas and transform to lowercase
            words_by_level[level] += [line.split()[0]] #get the first occurency in line as word
            
        else:
            pass
    
    return words_by_level


content = []
oxford_words = pd.DataFrame(columns=['word', 'level', 'source', 'type'])

# load all pdfs
for dirname, _, filenames in os.walk(OXFORD_PATH):
    for filename in filenames:
        print(filename)
        reader = PdfReader(f'{OXFORD_PATH}/{filename}')
        for page in reader.pages:
            content += page.extract_text().splitlines()
        processed = process_pdf(content)
        for level in processed:
            dict_type = 'us' if 'American' in filename else 'uk'
            oxford_words = pd.concat([oxford_words,
                                      pd.DataFrame({'word'   : processed[level],
                                                    'level'  : level,
                                                    'source' : filename,
                                                    'type'   : dict_type
                                                   })
                                     ])

# sort words and keep unique with higher levels
oxford_words = oxford_words.sort_values(by=['word', 'level'], ascending=True)
oxford_words = oxford_words.drop_duplicates(subset=['word'], keep='last')

# после сортировки и удаления дубликатов остался только американский инглиш
# похоже там все уровни слов считаются выше

oxford_words.groupby('level') \
            .agg({'word':'count', 'source': 'unique'}) \
            .style.format({'word':'{:,.0f} слов'})


American_Oxford_3000_by_CEFR_level.pdf
The_Oxford_3000_by_CEFR_level.pdf
The_Oxford_5000_by_CEFR_level.pdf
American_Oxford_5000_by_CEFR_level.pdf


,word,source
level,,
A1,735 слов,['American_Oxford_5000_by_CEFR_level.pdf']
A2,747 слов,['American_Oxford_5000_by_CEFR_level.pdf']
B1,758 слов,['American_Oxford_5000_by_CEFR_level.pdf']
B2,"1,446 слов",['American_Oxford_5000_by_CEFR_level.pdf']
C1,"1,198 слов",['American_Oxford_5000_by_CEFR_level.pdf']


In [70]:
oxford_words


,word,level,source,type
887,a,A1,American_Oxford_5000_by_CEFR_level.pdf,us
716,abandon,B2,American_Oxford_5000_by_CEFR_level.pdf,us
856,ability,A2,American_Oxford_5000_by_CEFR_level.pdf,us
857,able,A2,American_Oxford_5000_by_CEFR_level.pdf,us
1109,abolish,C1,American_Oxford_5000_by_CEFR_level.pdf,us
...,...,...,...,...
1714,yours,A2,American_Oxford_5000_by_CEFR_level.pdf,us
1773,yourself,A1,American_Oxford_5000_by_CEFR_level.pdf,us
1594,youth,B1,American_Oxford_5000_by_CEFR_level.pdf,us
1715,zero,A2,American_Oxford_5000_by_CEFR_level.pdf,us


In [54]:
dump(oxford_words,'oxford_dictionary.joblib')

['oxford_dictionary.joblib']

## Subtitles .srt

In [68]:
# parse and clean subtitles   
def clean_text(content):
    text = []
    for line in content.splitlines():                        
        if re.search(r'[A-Za-z]',line):                      # get non empty lines with words
            line = line.lower()                              # transform to lowercase
            line = re.sub(r'\<[^<]+?\>', ' ', line)          # remove html tags
            line = re.sub(r'(\(.+\))|(\[.+\])', ' ', line)   # remove actions in parenthesis
            line = re.sub(r'([\w#\s]+\:)', ' ', line)        # remove speaker in dialogs
            line = re.sub(r'(?<=\w)\'(?=\w)', '', line)      # glue apostroph inside words
            line = re.sub(r'[^a-z]', ' ', line)              # remove non-word symbols
#             line = re.sub(r'\b\w{1,2}\b', '', line)          # remove single and two symbol sequences
            line = re.sub(r'\s\s+', ' ', line).strip()       # remove extra spaces longer than single
            if len(line)>0:
                text.append(line)
    return ' '.join(text)


# process file
def process_srt(dirname, filename):
    global count
    if not filename.endswith('.srt'):                        # skip non srt files
        print('посторонний файл', filename)
        return '', False
    fullpath = os.path.join(dirname,filename)
    try:
        file = open(fullpath, 'rb')
        content = file.read()
        file.close()
        encoding = chardet.detect(content)['encoding']      # detect encoding and convert
#         print(encoding)
        content = content.decode(encoding)
    except:
        print('не прочиталось', filename)
        return '', False, 
    return content, clean_text(content)                     # clean text and return

movies = pd.DataFrame(columns=['filename', 'raw', 'content', 'level', 'lemmas'] + ENGLISH_LEVELS + \
                              [l+'uk' for l in ENGLISH_LEVELS] + \
                              [l+'us' for l in ENGLISH_LEVELS] + \
                              [l+'ratio' for l in ENGLISH_LEVELS]
                     )

# recursive walk through dirs
for dirname, _, filenames in os.walk(SUBTITLES_PATH):
    for filename in filenames:
        level  = dirname.split('/')[-1]                       # get level name from dir
        raw, subs = process_srt(dirname, filename)            # process file
        if subs:                                              # add movie to dataframe
            movies.loc[len(movies)] = \
                {'filename': filename.replace('.srt', ''),
                 'raw'     : raw,
                 'content' : subs,
                 'level'   : level
                }

movies.head()

посторонний файл .DS_Store
посторонний файл .DS_Store


,filename,raw,content,level,lemmas,A1,A2,B1,B2,C1,...,A1us,A2us,B1us,B2us,C1us,A1ratio,A2ratio,B1ratio,B2ratio,C1ratio
0,"Crown, The S01E01 - Wolferton Splash.en","1\r\n00:00:59,400 --> 00:01:03,960\r\nIn seeki...",in seeking his british nationalization his roy...,B2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Suits.Episode 1- Denial,"1\r\n00:00:07,298 --> 00:00:09,332\r\nYou're t...",youre the most amazing woman i have ever met a...,B2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Crazy4TV.com - Suits.S06E06.720p.BluRay.x265.H...,"1\r\n00:00:00,877 --> 00:00:02,337\r\n(HARVEY ...",ive been after sutter for three years now this...,B2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Suits.S02E08.HDTV.x264-EVOLVE,"1\r\n00:00:05,640 --> 00:00:07,641\r\n[Car hor...",youre late nope seconds early good learning to...,B2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Virgin.River.S01E07.INTERNAL.720p.WEB.x264-STRiFE,"1\r\n00:00:10,468 --> 00:00:13,178\r\nAre you ...",are you sure i cant convince you to stay no yo...,B2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Excel labels processing

In [5]:
movie_labels = pd.read_excel(f'{SCORES_PATH}/movies_labels.xlsx', index_col='id')
movie_labels.info()
movie_labels.columns = ['movie', 'level']
print('\nДубликаты полные', movie_labels.duplicated().sum())
print('Дубликаты в названиях фильмов', movie_labels.movie.duplicated().sum())
movie_labels = movie_labels.drop_duplicates()
movie_labels


<class 'pandas.core.frame.DataFrame'>
Int64Index: 241 entries, 0 to 240
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Movie   241 non-null    object
 1   Level   241 non-null    object
dtypes: object(2)
memory usage: 5.6+ KB

Дубликаты полные 2
Дубликаты в названиях фильмов 4


,movie,level
id,,
0,10_Cloverfield_lane(2016),B1
1,10_things_I_hate_about_you(1999),B1
2,A_knights_tale(2001),B2
3,A_star_is_born(2018),B2
4,Aladdin(1992),A2/A2+
...,...,...
236,Matilda(2022),C1
237,Bullet train,B1
238,Thor: love and thunder,B2


In [6]:
# correct some mistakes in movies names
movie_labels.movie = movie_labels.movie.str.replace('.srt', '', regex=False)
movie_labels.loc[movie_labels.movie == 'Up (2009)', 'movie'] = 'Up(2009)'
movie_labels.loc[movie_labels.movie == 'The Grinch', 'movie'] = 'The.Grinch'

# удалим из level лишние символы, разделители оставим пробел
# из мультиуровней выберем наименьший
movie_labels.level = movie_labels.level \
                                 .str.replace(',', '', regex=False) \
                                 .str.replace('+', '', regex=False) \
                                 .str.replace('/', ' ', regex=False) \
                                 .str.split().transform(lambda x: min(x))

movie_labels.groupby('level').count().style.format({'movie':'{:.0f} фильмов'})

,movie
level,
A2,37 фильмов
B1,61 фильмов
B2,101 фильмов
C1,40 фильмов


In [44]:
movies[movies.filename.str.contains('ear')]

,movie,filename,content,level,lemmas,A1,A2,B1,B2,C1,...,coverage_more_than_B1ratio,coverage_less_equal_B2,coverage_less_equal_B2ratio,coverage_more_than_B2,coverage_more_than_B2ratio,coverage_less_equal_C1,coverage_less_equal_C1ratio,coverage_more_than_C1,coverage_more_than_C1ratio,target
185,0.0,Braveheart(1995),i shall tell you of william wallace historians...,B2,I shall tell you of william wallace historians...,278,176,152,150,66.0,...,0.081694,756,0.205100,66,0.055092,822,0.168305,0,0,4
191,0.0,The_Devil_Wears_Prad,good luck hi i have an appointment with emily ...,B2,good luck hi I have an appointment with emily ...,414,219,172,176,68.0,...,0.092284,981,0.266142,68,0.056761,1049,0.214783,0,0,4


In [7]:
# excel processing
for row in movie_labels.itertuples():

    n = movies.loc[movies.filename.str.contains(row.movie, regex=False)].shape[0]

    if n == 0:
        print('не найден текст', row.movie)

    elif n == 1:
        selected_movie_level = movies.loc[
            movies.filename.str.contains(row.movie, regex=False), 'level'].values[0]

        if selected_movie_level == 'Subtitles':         # replace Subtitles with excel level
             movies.loc[
                 movies.filename.str.contains(row.movie, regex=False), 'level'] = row.level

        elif selected_movie_level != row.level:          # replace with min current level or excel
             movies.loc[
                 movies.filename.str.contains(row.movie, regex=False), 'level'
             ] = min(selected_movie_level, row.level)

    else:
        print('не единственный текст', row.movie)
            
# movies

не найден текст The Secret Life of Pets.en
не найден текст Glass Onion
не найден текст Matilda(2022)
не найден текст Bullet train
не найден текст Thor: love and thunder
не найден текст Lightyear


## Lemmatization

In [8]:
%%time
# movies content lemmatization
movies['lemmas'] = movies.content.transform(lambda text: ' '.join([w.lemma_ for w in nlp(text)]))

CPU times: user 8min 53s, sys: 15.8 s, total: 9min 9s
Wall time: 9min 12s


## Feature engineering

In [9]:
# oxford words in movies lemmas
def count_words_by_level(row):

    row_lemmas = row.lemmas.split()
    
    total_score = oxford_words.loc[oxford_words.word.isin(row_lemmas)] \
                              .groupby('level').word.count().to_dict()
    
#     british_score = oxford_words.loc[
#         (oxford_words.type == 'uk') & \
#         (oxford_words.word.isin(row_lemmas))
#     ].groupby('level').word.count().to_dict()

#     american_score = oxford_words.loc[
#         (oxford_words.type == 'us') & \
#         (oxford_words.word.isin(row_lemmas))
#     ].groupby('level').word.count().to_dict()

#     total_score.update({k+'en':v for k,v in british_score.items()})
#     total_score.update({k+'us':v for k,v in american_score.items()})
    
    row[total_score.keys()] = total_score

    return row

movies = movies.apply(count_words_by_level, axis=1)


In [ ]:
%%time
# coverage of oxford dict and ratio of coverage        
def coverage_oxford(row):
    
    global l, oxford_equal, oxford_less, oxford_more
    row_lemmas = set(row.lemmas.split())
    
    cover_col = 'coverage_' + l
    ratio_col = 'coverage_' + l + 'ratio'
    
    cover_less = 'coverage_less_equal_' + l
    ratio_less = 'coverage_less_equal_' + l + 'ratio'

    cover_more = 'coverage_more_than_' + l
    ratio_more = 'coverage_more_than_' + l + 'ratio'
    
    # покрытие словаря в фильме (по уровням)
    row[cover_col] = sum(lemma in oxford_equal for lemma in row_lemmas)
    row[ratio_col] = row[cover_col] / len(oxford_equal)

    # покрытие словаря суммарно по уровням до заданного включительно
    row[cover_less] = sum(lemma in oxford_less for lemma in row_lemmas)
    row[ratio_less] = row[cover_less] / len(oxford_less)

    # покрытие словаря выше заданного уровня
    len_oxford_more = len(oxford_more)
    if len_oxford_more>0:
        row[cover_more] = sum(lemma in oxford_more for lemma in row_lemmas)
        row[ratio_more] = row[cover_more] / len(oxford_more)
    else:
        row[cover_more], row[ratio_more] = 0, 0

    return row

for l in ENGLISH_LEVELS:
    oxford_equal = oxford_words[oxford_words.level==l].word.values
    oxford_less  = oxford_words[oxford_words.level<=l].word.values
    oxford_more  = oxford_words[oxford_words.level>l].word.values
    movies = movies.apply(coverage_oxford, axis=1)


In [ ]:
# total lemmas count
# and unique lemmas count
# and unique ratio
movies['lemmas_count']  = movies.lemmas.transform(lambda x: len(x.split()))
movies['lemmas_unique'] = movies.lemmas.transform(lambda x: len(set(x.split())))
movies['lemmas_unique_ratio'] = movies.lemmas_unique / movies.lemmas_count

In [10]:
# ratio oxford words in content
for l in ENGLISH_LEVELS:
    movies[l+'ratio'] = movies[l]/movies.lemmas_unique

In [11]:
# count words by length
# and ratio to total count
for i in range(1,10):
    equal_col = 'equal_'+str(i)
    more_col  = 'more_than_'+str(i)
    less_col  = 'less_than_'+str(i)
    
    movies[equal_col] = movies.lemmas.transform(lambda x: sum(len(word)==i for word in x.split()))
    movies[more_col]  = movies.lemmas.transform(lambda x: sum(len(word)>i for word in x.split()))
    movies[less_col]  = movies.lemmas.transform(lambda x: sum(len(word)<i for word in x.split()))
    
    movies[equal_col + 'ratio'] = movies[equal_col] / movies.lemmas_count
    movies[more_col + 'ratio']  = movies[more_col] / movies.lemmas_count
    movies[less_col + 'ratio']  = movies[less_col] / movies.lemmas_count
    

In [307]:
# export data movies to csv
movies['target'] = movies.level.replace({'A1':1, 'A2':2, 'B1':3, 'B2':4, 'C1':5, 'C2':6})
movies = movies.fillna(0)
movies = movies[movies.level!='Subtitles']
movies.to_csv('movies_df.csv', index=False)

try:
    os.system('say "data processing completed"')
except:
    pass

# MODELS TEST

In [6]:
movies = pd.read_csv('movies_df.csv')


In [7]:
text = 'lemmas'

lemmas_num = ['lemmas_count', 'lemmas_unique', 'lemmas_unique_ratio']

levels_ratio = [l+'ratio' for l in ENGLISH_LEVELS]
# levels_us    = [l+'us' for l in ENGLISH_LEVELS]
# levels_uk    = [l+'uk' for l in ENGLISH_LEVELS]

length_equal = ['equal_'+str(i) for i in range(1,10)]
length_more  = ['more_than_'+str(i) for i in range(1,10)]
length_less  = ['less_than_'+str(i) for i in range(1,10)]

equal_ratio = [c+'ratio' for c in length_equal]
more_ratio  = [c+'ratio' for c in length_more]
less_ratio  = [c+'ratio' for c in length_less]

oxford_cover = ['coverage_' + l for l in ENGLISH_LEVELS]
oxford_ratio = ['coverage_' + l + 'ratio' for l in ENGLISH_LEVELS]
    
oxford_cover_less= ['coverage_less_equal_' + l for l in ENGLISH_LEVELS]
oxford_ratio_less= ['coverage_less_equal_' + l + 'ratio' for l in ENGLISH_LEVELS]

oxford_cover_more= ['coverage_more_than_' + l for l in ENGLISH_LEVELS]
oxford_ratio_more= ['coverage_more_than_' + l + 'ratio' for l in ENGLISH_LEVELS]


# num_us = lemmas_num + levels_us
# num_en = lemmas_num + levels_uk

num = [*lemmas_num,
       *levels_ratio,
#        *length_equal,
#        *length_more,
#        *length_less,
       *equal_ratio,
       *more_ratio,
       *less_ratio,
#        *oxford_cover,
       *oxford_ratio,
#        *oxford_cover_less,
       *oxford_ratio_less,
#        *oxford_cover_more,
       *oxford_ratio_more
      ]

target = 'level'


In [8]:
encoder = OrdinalEncoder()

X_train, X_test, y_train, y_test = train_test_split(movies, 
                                                    movies.target,
#                                                     encoder.fit_transform(movies[[target]]), 
                                                    stratify=movies[[target]],
                                                    random_state=RANDOM_STATE,
                                                    test_size=0.3
                                                   )


In [13]:
# vectorizer = TfidfVectorizer()
vectorizer = CountVectorizer()
scaler     = StandardScaler()

preprocessor = ColumnTransformer([
    ('num', scaler, more_ratio),
    ('txt', vectorizer,  text)
], remainder='drop')


## Regressor

In [14]:
%%time

params = {
#     'pre__txt__stop_words'  : [None, 'english'],
#     'pre__txt__ngram_range' : [(1,1), (1,2), (1,3)],
#     'pre__txt__max_features': [None, 1000],
#     'pre__txt__min_df'      : [1, 10, 100],
    'reg__learning_rate'    : [0.05, 0.1, 0.2],
    'reg__n_estimators'     : [50, 100, 200],
    'reg__max_depth'        : [3, 5, 7],
    'reg__min_child_weight' : [1, 3, 5],
    'reg__gamma'            : [0, 0.1, 0.5, 1],
    'reg__subsample'        : [0.6, 0.8, 1.0],
    'reg__colsample_bytree' : [0.6, 0.8, 1.0],
    'reg__reg_alpha'        : [0, 0.1, 1],
    'reg__reg_lambda'       : [0, 0.1, 1],
    'reg__gamma'            : [0, 1, 5]
}

regressor = xgboost.XGBRegressor(random_state=RANDOM_STATE)

pipeline = Pipeline(steps=[
    ('pre', preprocessor),
    ('reg', regressor)
])

rc = RandomizedSearchCV(pipeline,
                        param_distributions=params,
                        scoring='neg_mean_squared_error',
                        error_score='raise',
                        random_state=RANDOM_STATE,
                        n_jobs=-1
                       )

rc.fit(X_train,y_train)

try:
    os.system('say "Model training completed"')
except:
    pass

-rc.best_score_

CPU times: user 3.05 s, sys: 1.07 s, total: 4.12 s
Wall time: 25.4 s


0.40931243274889384

In [20]:
rc

RandomizedSearchCV(error_score='raise',
                   estimator=Pipeline(steps=[('pre',
                                              ColumnTransformer(transformers=[('num',
                                                                               StandardScaler(),
                                                                               ['more_than_1ratio',
                                                                                'more_than_2ratio',
                                                                                'more_than_3ratio',
                                                                                'more_than_4ratio',
                                                                                'more_than_5ratio',
                                                                                'more_than_6ratio',
                                                                                'more_than_7ratio',
                                                                                'more_than_8ratio',
                                                                                'more_than_9ratio']),
                                                                              ('txt',
                                                                               CountVectorizer(),
                                                                               'lemmas')])),
                                             ('reg',
                                              XGB...
                   n_jobs=-1,
                   param_distributions={'reg__colsample_bytree': [0.6, 0.8,
                                                                  1.0],
                                        'reg__gamma': [0, 1, 5],
                                        'reg__learning_rate': [0.05, 0.1, 0.2],
                                        'reg__max_depth': [3, 5, 7],
                                        'reg__min_child_weight': [1, 3, 5],
                                        'reg__n_estimators': [50, 100, 200],
                                        'reg__reg_alpha': [0, 0.1, 1],
                                        'reg__reg_lambda': [0, 0.1, 1],
                                        'reg__subsample': [0.6, 0.8, 1.0]},
                   random_state=7, scoring='neg_mean_squared_error')

In [21]:
dump(rc, 'model_english_level.joblib')


['model_english_level.joblib']

In [18]:
feature_names = rc.best_estimator_[0].get_feature_names_out()
feature_importances = rc.best_estimator_[-1].feature_importances_
fi = pd.DataFrame({'feature':feature_names, 'weight': feature_importances})
fi.feature = fi.feature.str[5:]
# fi['type'] = fi.feature.str[:3]

fi.sort_values(by='weight', ascending=False).head(50)

,feature,weight
7463,firm,0.054635
9137,harvey,0.041164
18220,shh,0.032929
3899,come,0.027545
7639,fly,0.026115
5,more_than_6ratio,0.024185
22339,which,0.024103
9567,honest,0.023404
1,more_than_2ratio,0.020367
6401,eld,0.016603
